In [1]:
import pandas as pd
import psycopg2
import numpy as np



from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
from selenium.webdriver.support.ui import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service



from tqdm import tqdm
tqdm.pandas()
import time



from bs4 import BeautifulSoup
import re

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders


In [2]:
s = Service("../chromedriver_win32/chromedriver.exe")

In [4]:
class postgres_conn:
    def getConn(self):
        try:
            connection = psycopg2.connect(user='postgres',
                                          password='postgres_007',
                                          host="1.pgsql.db.1digitalstack.com",
                                          port='5432',
                                          database='postgres')

            cursor = connection.cursor()
            # Print PostgreSQL Connection properties
            print(connection.get_dsn_parameters(), "\n")

            # Print PostgreSQL version
            return cursor, connection

        except (Exception, psycopg2.Error) as error:
            print("Error while connecting to PostgreSQL", error)
            return error, error
    def getConn2(self):
        try:
            connection = psycopg2.connect(user='postgres',
                                          password='reporting_007',
                                          host="2.pgsql.db.1digitalstack.com",
                                          port='5432',
                                          database='postgres')

            cursor = connection.cursor()
            # Print PostgreSQL Connection properties
            print(connection.get_dsn_parameters(), "\n")

            # Print PostgreSQL version
            return cursor, connection

        except (Exception, psycopg2.Error) as error:
            print("Error while connecting to PostgreSQL", error)
            return error, error


    def close_connection(self, cursor, connection):
        # closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")
            
pg = postgres_conn()
conn = pg.getConn()
conn2 = pg.getConn2()

{'user': 'postgres', 'channel_binding': 'prefer', 'dbname': 'postgres', 'host': '1.pgsql.db.1digitalstack.com', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'disable', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

{'user': 'postgres', 'channel_binding': 'prefer', 'dbname': 'postgres', 'host': '2.pgsql.db.1digitalstack.com', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'disable', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 



In [8]:
job_id=input("Enter keystone ids with comma.").split(",")
days=input(" Enter no of days for which data needs brand mapping.\n")
job_id.append("")
tuple(job_id)

Enter keystone ids with comma.6467549e5958ed6c1f2798a6
 Enter no of days for which data needs brand mapping.
17


('6467549e5958ed6c1f2798a6', '')

In [9]:
query = f"""
select brand,title,sku,job_name,channel_id
from ready.serp_details smd join ready.serp_mongo_details m on m.serp_mongo_id=smd.serp_mongo_id
where smd.crawl_date > now() - interval '{days} day' and keystone_job_id in {tuple(job_id)}
"""
serp_details = pd.read_sql_query(query, conn[1])

C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_107052\2162843895.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  serp_details = pd.read_sql_query(query, conn[1])


In [10]:
serp_details

,brand,title,sku,job_name,channel_id
0,KLACK,Arcade console with 4000 classic and 7000 new ...,94399377,Mediamarket Request No. 466 serp_job,58
1,AGD,Video Game Console - Retro Mini Console with 6...,88879398,Mediamarket Request No. 466 serp_job,58
2,KLACK,"Video game console - PANDORABOX3DWIFI KLACK, 6...",104976396,Mediamarket Request No. 466 serp_job,58
3,KLACK,"Video game console - CONS400CONMANDO KLACK, 64...",104976395,Mediamarket Request No. 466 serp_job,58
4,MYARCADE,"Video Game - RETRO CHAMP NES MYARCADE CONSOLE,...",95940292,Mediamarket Request No. 466 serp_job,58
...,...,...,...,...,...
267,LOGITECH G,"Shifter - Logitech Driving Force Shifter, For ...",1292218,Mediamarket Request No. 466 serp_job,58
268,SEAGATE,2TB External Hard Drive - ‎Seagate STKX2000400...,1520256,Mediamarket Request No. 466 serp_job,58
269,SEAGATE,"4TB External Hard Drive - Seagate STKX4000402,...",1525870,Mediamarket Request No. 466 serp_job,58
270,SEAGATE,"External Hard Drive 8TB - Seagate STKW8000400,...",1525871,Mediamarket Request No. 466 serp_job,58


In [11]:
rawdf1=serp_details.drop_duplicates('sku')

df_ASIN = rawdf1[["sku"]]
df_ASIN.dropna(inplace=True)
df_ASIN.drop_duplicates(inplace=True)
asinlist = list(df_ASIN['sku'])
len(asinlist)

C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_107052\2610474507.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ASIN.dropna(inplace=True)
C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_107052\2610474507.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ASIN.drop_duplicates(inplace=True)


139

In [12]:
query_2 = f""" 
Select channel_sku_id,country_code, brand_name, categories,category_id
from entity.product_master
where channel_sku_id in {tuple(asinlist)} 
"""

pm_data = pd.read_sql_query(query_2, conn[1])
pm_data

C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_107052\3517360684.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pm_data = pd.read_sql_query(query_2, conn[1])


,channel_sku_id,country_code,brand_name,categories,category_id
0,102136213,ES,PDP,None,None
1,102136433,ES,ARDISTEL,None,None
2,104205773,ES,NINTENDO,None,None
3,104976395,ES,KLACK,None,None
4,104976396,ES,KLACK,None,None
...,...,...,...,...,...
134,95940275,ES,MY ARCADE,None,None
135,95940292,ES,MYARCADE,None,None
136,97970912,ES,INF,None,None
137,99138991,ES,UBI SOFT,None,None


In [13]:
mapped_df=pd.merge(rawdf1,pm_data, how='left', left_on=['sku'],right_on=['channel_sku_id'])
mapped_df

,brand,title,sku,job_name,channel_id,channel_sku_id,country_code,brand_name,categories,category_id
0,KLACK,Arcade console with 4000 classic and 7000 new ...,94399377,Mediamarket Request No. 466 serp_job,58,94399377,ES,KLACK,None,None
1,AGD,Video Game Console - Retro Mini Console with 6...,88879398,Mediamarket Request No. 466 serp_job,58,88879398,ES,AGD,None,None
2,KLACK,"Video game console - PANDORABOX3DWIFI KLACK, 6...",104976396,Mediamarket Request No. 466 serp_job,58,104976396,ES,KLACK,None,None
3,KLACK,"Video game console - CONS400CONMANDO KLACK, 64...",104976395,Mediamarket Request No. 466 serp_job,58,104976395,ES,KLACK,None,None
4,MYARCADE,"Video Game - RETRO CHAMP NES MYARCADE CONSOLE,...",95940292,Mediamarket Request No. 466 serp_job,58,95940292,ES,MYARCADE,None,None
...,...,...,...,...,...,...,...,...,...,...
134,WESTERN DIGITAL,External hard drive 12 TB - WD_BLACK D10 Game ...,1468216,Mediamarket Request No. 466 serp_job,58,1468216,ES,WESTERN DIGITAL,None,None
135,SEAGATE,2TB External Hard Drive - ‎Seagate STKX2000400...,1520256,Mediamarket Request No. 466 serp_job,58,1520256,ES,SEAGATE,None,None
136,SEAGATE,"4TB External Hard Drive - Seagate STKX4000402,...",1525870,Mediamarket Request No. 466 serp_job,58,1525870,ES,SEAGATE,None,None
137,SEAGATE,"External Hard Drive 8TB - Seagate STKW8000400,...",1525871,Mediamarket Request No. 466 serp_job,58,1525871,ES,SEAGATE,None,None


In [16]:
df2=mapped_df[mapped_df['categories'].isnull()][['channel_id','sku','title','brand_name']]
df2

,channel_id,sku,title,brand_name
0,58,94399377,Arcade console with 4000 classic and 7000 new ...,KLACK
1,58,88879398,Video Game Console - Retro Mini Console with 6...,AGD
2,58,104976396,"Video game console - PANDORABOX3DWIFI KLACK, 6...",KLACK
3,58,104976395,"Video game console - CONS400CONMANDO KLACK, 64...",KLACK
4,58,95940292,"Video Game - RETRO CHAMP NES MYARCADE CONSOLE,...",MYARCADE
...,...,...,...,...
134,58,1468216,External hard drive 12 TB - WD_BLACK D10 Game ...,WESTERN DIGITAL
135,58,1520256,2TB External Hard Drive - ‎Seagate STKX2000400...,SEAGATE
136,58,1525870,"4TB External Hard Drive - Seagate STKX4000402,...",SEAGATE
137,58,1525871,"External Hard Drive 8TB - Seagate STKW8000400,...",SEAGATE


In [14]:
df=pd.DataFrame(columns=['channel_sku_id','categories','seller_info','first_image'])
df

,channel_sku_id,categories,seller_info,first_image


In [17]:
asinlist = list(df2['sku'])
len(asinlist)

139

In [ ]:
request_no=input("Enter the request number of the requests seperated with underscore(_) ")
#547_552
file='final_output_mediamarket_spain_{}.xlsx'.format(request_no)

In [18]:
chrome_opt=Options()
chrome_opt.add_argument("--incognito")
driver = webdriver.Chrome(service=s,options=chrome_opt)
capabilities = driver.capabilities
chrome_driver_version = capabilities['chrome']['chromedriverVersion']

# Print the ChromeDriver version
print("ChromeDriver version:", chrome_driver_version)

ChromeDriver version: 116.0.5845.96 (1a391816688002153ef791ffe60d9e899a71a037-refs/branch-heads/5845@{#1382})


In [21]:
for i in tqdm(range(0,len(asinlist))):
    sku=asinlist[i]
    url="https://www.mediamarkt.es/es/product/-{}.html".format(sku)
    
    print('************************')
    print(i)
    print(url)
    driver.get(url)
    first_image=''
    bs=BeautifulSoup(driver.page_source,'html.parser')
    cat=[]
    try:
        element = driver.find_elements(By.XPATH, "//ol[@class='Breadcrumb-styled__StyledList-sc-57676b39-1 cXCHDb']//li")
        cat=[element[e].text for e in range(1,len(element))] 
        print(cat)
    except:
         print('cat not  found')
    try:
        image = bs.find('li',class_="sc-hAQmFe iGozss")
        first_image=image.find('img').get('src')
        print(first_image)
    except:
        print('image not found')

  0%|                                                                                          | 0/139 [00:00<?, ?it/s]

************************
0
https://www.mediamarkt.es/es/product/-94399377.html


  1%|▌                                                                                 | 1/139 [00:09<20:57,  9.11s/it]

['Consolas y videojuegos', 'Consolas y Juegos Retro']
https://assets.mmsrg.com/isr/166325/c1/-/ASSET_MP_94399380/fee_325_225_png
************************
1
https://www.mediamarkt.es/es/product/-88879398.html


  1%|█▏                                                                                | 2/139 [00:15<17:41,  7.74s/it]

...

['Accesorios de informática', 'Almacenamiento', 'Discos duros externos']
image not found
************************
137
https://www.mediamarkt.es/es/product/-1525871.html


 99%|███████████████████████████████████████████████████████████████████████████████▍| 138/139 [28:42<00:11, 11.92s/it]

['Accesorios de informática', 'Almacenamiento', 'Discos duros externos']
image not found
************************
138
https://www.mediamarkt.es/es/product/-1557296.html


100%|████████████████████████████████████████████████████████████████████████████████| 139/139 [28:55<00:00, 12.49s/it]

['Gaming', 'Accesorios gaming', 'Gamepads']
image not found


In [22]:
df

,channel_sku_id,categories,seller_info,first_image


In [ ]:
final1=pd.merge(rawdf1,df,left_on='sku', right_on='channel_sku_id', how= 'right')
final=final1[['sku','title','brand','categories','first_image']]

In [ ]:
final

In [ ]:
final.to_excel(file)

In [ ]:
email_user = 'alerts@1digitalstack.ai'
email_password = ''
email_send = ['ghansham.pindoriya@1digitalstack.ai','karan.kumar@1digitalstack.ai','arihant.singh@1digitalstack.ai','sambhav.jain@1digitalstack.ai','ankur.dwivedi@1digitalstack.ai','kinshu@1digitalstack.ai']

subject = file
msg = MIMEMultipart()
msg['From'] = email_user
#msg['From'] = "'Abhishek' <{email_user}>"
msg['To'] = ', '.join(email_send)
msg['Subject'] = subject

body = 'Hi,\n\nPFA of today''s mapping request.Please reply as done as you pick this job.'
msg.attach(MIMEText(body,'plain'))

filename=[file]

for file in filename:
    attachment  =open(file,'rb')
    part = MIMEBase('application','octet-stream')
    part.set_payload((attachment).read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition',"attachment; filename= "+file)
    msg.attach(part)
    text = msg.as_string()

server = smtplib.SMTP('1digitalstack.com',25)
#server.login(email_user,email_password)

server.sendmail(email_user,email_send,text)
server.quit()